In [ ]:
# Optional link google drive, copy the code and enter.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Setup

In [ ]:
!pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /tmp/pip-install-9c17l7k9/twint_90871c991f7b4a54bde7a2b1ad3ea5f8
  Running command git clone -q https://github.com/twintproject/twint.git /tmp/pip-install-9c17l7k9/twint_90871c991f7b4a54bde7a2b1ad3ea5f8
  Running command git checkout -q origin/master
     |████████████████████████████████| 1.3 MB 5.3 MB/s 
     |████████████████████████████████| 263 kB 60.9 MB/s 
     |████████████████████████████████| 363 kB 26.1 MB/s 
     |████████████████████████████████| 291 kB 56.8 MB/s 
     |████████████████████████████████| 294 kB 53.0 MB/s 
     |████████████████████████████████| 142 kB 51.3 MB/s 
  Created wheel for twint: filename=twint-2.1.21-py3-none-any.whl size=38870 sha256=10b892ea271ae7c2e65f91000d1fbd56051e7893f6ea79a87ea042e5e2fdd2f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-h5swqvan/wheels/8d/dc/9f/74b4483d5f997036f04aec7f42bd4b3c80f04264920c368068
  Created wheel for fake-useragent: filename

In [ ]:
!pip install alpaca-trade-api

     |████████████████████████████████| 43 kB 906 kB/s 
     |████████████████████████████████| 636 kB 9.7 MB/s 
     |████████████████████████████████| 1.3 MB 37.9 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 103 kB 63.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:


In [ ]:
!pip3 install python-dotenv

In [ ]:
import os
import re
import io
import string
import datetime
import numpy as np
import twint
import nest_asyncio
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.backend import manual_variable_initialization

from dotenv import load_dotenv, find_dotenv
import alpaca_trade_api as tradeapi

import requests

nest_asyncio.apply()
manual_variable_initialization(True)

# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

ModuleNotFoundError: ignored

In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  
  labels = dataframe.pop('target')
  
  features_dataset = tf.data.Dataset.from_tensor_slices(dataframe)
  labels_dataset = tf.data.Dataset.from_tensor_slices(labels)

  ds = tf.data.Dataset.zip((features_dataset, labels_dataset))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

# A utility method to vectorize our tweets labeled dataset
def int_vectorize_text(text, label):
  return encoder(text), label

AUTOTUNE = tf.data.AUTOTUNE

# Configure for model
def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Print prediction examples
def print_my_examples(inputs, results):
  if isinstance(inputs, pd.Series):
    inputs.reset_index(inplace=True)
  result_for_printing = \
    [f'input: {inputs[i]:<30}\nscore: {results[i][0]:.6f}\n'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()

In [ ]:
# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 140

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
encoder = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [ ]:
# Read the twitter sentiment data
twitter_df = pd.read_csv(
    './gdrive/MyDrive/twitter-sentiment/twitter.csv',
    encoding ="ISO-8859-1",
    names = ["target", "ids", "date", "flag", "user", "text"])
twitter_df = twitter_df.drop(columns=['ids', 'date', 'flag', 'user'])

# In the original dataset "4" indicates the pet was not adopted.
twitter_df['target'] = np.where(twitter_df['target']==4, 0, 1)

In [ ]:
# View head and target counts
display(twitter_df.head())
twitter_df.target.value_counts()

In [ ]:
# Create train/val/test splits
train, test = train_test_split(twitter_df, test_size=0.2, random_state=3)
train, val = train_test_split(twitter_df, test_size=0.2, random_state=3)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [ ]:
BATCH_SIZE = 64

raw_train_ds = df_to_dataset(train, shuffle=False, batch_size=BATCH_SIZE)
raw_val_ds = df_to_dataset(val, shuffle=False, batch_size=BATCH_SIZE)
raw_test_ds = df_to_dataset(test, shuffle=False, batch_size=BATCH_SIZE)

In [ ]:
for text, target in raw_train_ds.take(1):
  for i in range(5):
    print('text: ', text[i].numpy())
    print('target: ', target[i].numpy())
    print()

In [ ]:
# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = raw_train_ds.map(lambda text, target: text)
encoder.adapt(text_ds)

In [ ]:
# Review the vocab
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

# Review twitter encoded/decoded data
for text, label in raw_train_ds.take(1):
  for i in range(5):
    encoded_example = encoder(text.numpy()[i])
    print("Original:\t", text.numpy()[i])
    print("Round-trip:\t", " ".join(vocab[encoded_example]))
    print()

In [ ]:
train_ds = configure_dataset(raw_train_ds.map(int_vectorize_text))
val_ds = configure_dataset(raw_val_ds.map(int_vectorize_text))
test_ds = configure_dataset(raw_test_ds.map(int_vectorize_text))

## Base Model

In [ ]:
# Base model setup
embedding_dim = 16
date_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = f'./gdrive/MyDrive/twitter-sentiment/base-model/logs/{date_time}'
model_save_dir = f'./gdrive/MyDrive/twitter-sentiment/base-model/model/{date_time}'
embedding_dir = '/content/gdrive/MyDrive/twitter-sentiment/base-model/embeddings/' + date_time

os.mkdir(embedding_dir)

model_basic = Sequential([
  Embedding(vocab_size, embedding_dim, name='embedding'),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [ ]:
# Setup tensorboard logs
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

In [ ]:
# Compile base model
model_basic.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# tf.keras.metrics.FalseNegatives()
# tf.keras.metrics.FalsePositives()
# tf.keras.metrics.TrueNegatives()
# tf.keras.metrics.TruePositives()
# tf.keras.metrics.Precision()
# tf.keras.metrics.Recall()

In [ ]:
# View summary
model_basic.summary()

In [ ]:
# Fit base model
model_basic.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[tensorboard_callback])



In [ ]:
print("Evaluate")
result = model_basic.evaluate(test_ds)
dict(zip(model_basic.metrics_names, result))

In [ ]:
# Save base model
model_basic.save(model_save_dir)

In [ ]:
# View tensorboard saved info
%load_ext tensorboard
%tensorboard --logdir ./gdrive/MyDrive/twitter-sentiment/base-model/logs

In [ ]:
# Save weights to view on tensorflow projector
weights = model_basic.get_layer('embedding').get_weights()[0]
vocab = encoder.get_vocabulary()

# http://projector.tensorflow.org/
out_v = io.open(f'{embedding_dir}/vectors.tsv', 'w', encoding='utf-8')
out_m = io.open(f'{embedding_dir}/metadate.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

## RNN Model

In [ ]:
# RNN model setup
embedding_dim = 16
date_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = f'./gdrive/MyDrive/twitter-sentiment/rnn-model/logs/{date_time}'
model_save_dir = f'./gdrive/MyDrive/twitter-sentiment/rnn-model/model/{date_time}'
embedding_dir = '/content/gdrive/MyDrive/twitter-sentiment/rnn-model/embeddings/' + date_time

os.mkdir(embedding_dir)

# RNN model setup
model_rnn = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=64,
        name='embedding',
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
# Setup tensorboard logs
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

In [ ]:
# Compile RNN model
model_rnn.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
# View summary
model_rnn.summary()

In [ ]:
# Fit RNN model
history = model_rnn.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tensorboard_callback])

In [ ]:
print("Evaluate")
result = model_rnn.evaluate(test_ds)
dict(zip(model_rnn.metrics_names, result))

In [ ]:
# Save RNN model
model_rnn.save(model_save_dir)

In [ ]:
# View tensorboard saved info
%reload_ext tensorboard
%tensorboard --logdir ./gdrive/MyDrive/twitter-sentiment/rnn-model/logs

In [ ]:
# Save weights to view on tensorflow projector
weights = model_rnn.get_layer('embedding').get_weights()[0]
vocab = encoder.get_vocabulary()

# http://projector.tensorflow.org/
out_v = io.open(f'{embedding_dir}/vectors.tsv', 'w', encoding='utf-8')
out_m = io.open(f'{embedding_dir}/metadate.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

## RNN 2-layer Model

In [ ]:
# RNN model setup
embedding_dim = 16
date_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = f'./gdrive/MyDrive/twitter-sentiment/rnn2-model/logs/{date_time}'
model_save_dir = f'./gdrive/MyDrive/twitter-sentiment/rnn2-model/model/{date_time}'
embedding_dir = '/content/gdrive/MyDrive/twitter-sentiment/rnn2-model/embeddings/' + date_time

os.mkdir(embedding_dir)

# RNN 2-layer model setup
model_rnn_2_layer = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True, name='embedding'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
# Setup tensorboard logs
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

In [ ]:
# Compile RNN 2-layer model
model_rnn_2_layer.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
# View summary
model_rnn_2_layer.summary()

In [ ]:
# Fit RNN 2-layer model
history = model_rnn_2_layer.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[tensorboard_callback])

In [ ]:
print("Evaluate")
result = model_rnn2.evaluate(test_ds)
dict(zip(model_rnn2.metrics_names, result))

In [ ]:
# Save RNN 2-layer model
model_rnn_2_layer.save(model_save_dir)

In [ ]:
# View tensorboard saved info
%reload_ext tensorboard
%tensorboard --logdir ./gdrive/MyDrive/twitter-sentiment/rnn2-model/logs

In [ ]:
# Save weights to view on tensorflow projector
weights = model_rnn_2_layer.get_layer('embedding').get_weights()[0]
vocab = encoder.get_vocabulary()

# http://projector.tensorflow.org/
out_v = io.open(f'{embedding_dir}/vectors.tsv', 'w', encoding='utf-8')
out_m = io.open(f'{embedding_dir}/metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
# View tensorboard saved info
%reload_ext tensorboard
%tensorboard --logdir ./gdrive/MyDrive/twitter-sentiment/rnn2-model/logs

## Load Models Back for Predictions

In [ ]:
# text:  [b'@umeshunni yeah iPhone does have more going for it.. now thats decided.. all thats left is wait for 3G here.. ']
# target:  1

# text:  [b'@BenjaminHouston oh I wish you guys were coming here  I wanna see savvy!!! ( and you guys of course) ha']
# target:  1

# text:  [b'@CaliSmiles06 hmm. ;) yuh, up &amp; out with the #rat boys &amp; girls. ']
# target:  0

# text:  [b'never thought my daughter would want to borrow my clothes... even less that she would suggest that i borrow hers... luckily it didnt fit ']
# target:  0

# text:  [b"@xxashwee25  i can't stop touching it cause its so thin! when i put it in a pony tail is like soo tiny compared to usual."]
# target:  0
examples = ['@umeshunni yeah iPhone does have more going for it.. now thats decided.. all thats left is wait for 3G here.. ',
            '@BenjaminHouston oh I wish you guys were coming here  I wanna see savvy!!! ( and you guys of course) ha',
            '@CaliSmiles06 hmm. ;) yuh, up &amp; out with the #rat boys &amp; girls. ',
            'never thought my daughter would want to borrow my clothes... even less that she would suggest that i borrow hers... luckily it didnt fit ',
            "@xxashwee25  i can't stop touching it cause its so thin! when i put it in a pony tail is like soo tiny compared to usual."]

In [ ]:
model_basic_reloaded = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/base-model/model/' + os.listdir('./gdrive/MyDrive/twitter-sentiment/base-model/model')[-1])
model_rnn_reloaded = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/rnn-model/model/' + os.listdir('./gdrive/MyDrive/twitter-sentiment/rnn-model/model')[-1])
mode_rnn_2_layer_reloaded = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/rnn2-model/model/' + os.listdir('./gdrive/MyDrive/twitter-sentiment/rnn2-model/model')[-1])

In [ ]:
predictions = tf.sigmoid(model_basic_reloaded(tf.constant(encoder(examples))))
print_my_examples(examples, predictions)

## Load Models for Twitter

In [ ]:
# most_recent_base_model = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/base-model/model/' + os.listdir('./gdrive/MyDrive/twitter-sentiment/base-model/model')[-1])
best_base_model = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/base-model/model/20210913-035233')
# test_base_model = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/base-model/model/20210910-181817')

# most_recent_rnn_model = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/rnn-model/model/' + os.listdir('./gdrive/MyDrive/twitter-sentiment/rnn-model/model')[-1])
best_rnn_model = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/rnn-model/model/20210913-043938')
# test_rnn_model = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/rnn-model/model/20210910-181817')

# most_recent_rnn2_model = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/rnn2-model/model/' + os.listdir('./gdrive/MyDrive/twitter-sentiment/rnn2-model/model')[-1])
best_rnn2_model = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/rnn2-model/model/20210913-064343')
# test_rnn2_model = keras.models.load_model(f'/content/gdrive/MyDrive/twitter-sentiment/rnn2-model/model/20210910-181817')

In [ ]:
current_working_model = best_rnn2_model

## Add ATVI Twitter Data

In [ ]:
  # # Configure
  # for i in range(len(df)-1):
  #   date_time = df.loc[i,'Date']
  #   c = twint.Config()
  #   c.Search = "#blizzard"
  #   c.Since = df.loc[i,'Date'].strftime("%Y-%m-%d")
  #   c.Until = df.loc[i+1,'Date'].strftime("%Y-%m-%d")
  #   c.Store_csv = True
  #   c.Lang = "en"
  #   c.Output = f'./gdrive/MyDrive/twitter-sentiment/{c.Search}-{date_time}.csv'

  #   # Run
  #   try:
  #     twint.run.Search(c)
  #   catch:


In [ ]:
pulled_tweets = pd.read_csv('/content/gdrive/MyDrive/twitter-sentiment/combined_csv.csv', index_col='created_at', parse_dates=True, infer_datetime_format=True)
pulled_tweets.reset_index(inplace=True)

pulled_tweets

In [ ]:
# Pull only english tweets
pulled_tweets = pulled_tweets[pulled_tweets['language']=='en']
pulled_tweets.reset_index()
pulled_tweets = pulled_tweets.set_index('created_at')
pulled_tweets


In [ ]:
# Add predictions to pulled_tweets df
predictions = tf.sigmoid(current_working_model.predict(tf.constant(encoder(pulled_tweets['tweet']))))
predictions_list = []
for prediction in predictions:
  predictions_list.append(tf.keras.backend.get_value(prediction[0]))
pulled_tweets['prediction'] = predictions_list
pulled_tweets.head()


In [ ]:
# Create new dataframe of signals we want to include


signals = pulled_tweets[['tweet', 'prediction']].copy()


# signals = pulled_tweets[['tweet', 'prediction']].copy()


display(signals.head())

def center_prediction(prediction):
  if prediction == 0.5:
    return 0
  elif prediction > 0.5:
    return (prediction - 0.5)
  else:
    return -(.5-prediction)

# Cleanup prediction column
signals['prediction'] = signals['prediction'].apply(lambda x : center_prediction(x))
#signals.set_index('created_at')

In [ ]:
# Groupby date and just do a simple sum.....
# Could do it this way, i.e. with just a simple sum or by creating a threshold of the prediciton and then label each as 1 or 0
short_window = 12
long_window = 26
signal_window = 9

#signals = pd.DataFrame(signals.resample('D', on='created_at').prediction.sum())
signals = pd.DataFrame(signals.resample('D').prediction.sum())
# signals[f'prediction-{short_window}'] = signals['prediction'].rolling(short_window).mean()
# signals[f'prediction-{long_window}'] = signals['prediction'].rolling(long_window).mean()

# MACD calc
exp1 = signals['prediction'].ewm(span=12, adjust=False).mean()
exp2 = signals['prediction'].ewm(span=26, adjust=False).mean()
signals['MACD'] = macd = exp1 - exp2
signals['signal'] = exp3 = macd.ewm(span=signal_window, adjust=False).mean()

In [ ]:
signals = signals.dropna()
signals

In [ ]:
# Changed the labels on the graph to more accurately What was being graphed

p = figure(
    title="Prediction Daily",
    x_axis_label='Date',
    x_axis_type='datetime',
    y_axis_label='Prediction sum',
    plot_width=1000,
    plot_height=400)
p.line(signals.index, signals['MACD'], legend_label=f"{short_window} - {long_window} Day Rolling MACD", line_color='green')
p.line(signals.index, signals['signal'], legend_label=f"{signal_window} Day Rolling Signal Line")
show(p)

## Add NASDAQ 100 Twitter Data



In [ ]:
# nasdaq_tweets = pd.read_csv('./gdrive/MyDrive/twitter-sentiment/nasdaq100-tickers-combined.csv', 
#                             parse_dates=True, 
#                             infer_datetime_format=True
#                             )

In [ ]:
# display(nasdaq_tweets.head())
# display(nasdaq_tweets.info())

In [ ]:
# # Pull only english tweets
# nasdaq_tweets_en = nasdaq_tweets[nasdaq_tweets['Tweet language (ISO 639-1)']=='en']

In [ ]:
# # Add predictions to pulled_tweets df
# predictions_nasdaq = tf.sigmoid(current_working_model.predict(tf.constant(encoder(nasdaq_tweets_en['Tweet content']))))
# nasdaq_predictions_list = []
# for prediction in predictions_nasdaq:
#   nasdaq_predictions_list.append(tf.keras.backend.get_value(prediction[0]))
# nasdaq_tweets_en['prediction'] = nasdaq_predictions_list

In [ ]:
# nasdaq_tweets_en.head()

In [ ]:
# nasdaq_tweets_en.to_csv(f'/content/gdrive/MyDrive/twitter-sentiment/nasdaq-tweets-cleaned-rnn2-20210913-064343', index=False)

In [ ]:
cleaned_nasdaq_predictions = pd.read_csv(
    '/content/gdrive/MyDrive/twitter-sentiment/nasdaq-tweets-cleaned-rnn2-20210913-064343',
    parse_dates=True,
    infer_datetime_format=True,
    )

In [ ]:
cleaned_nasdaq_predictions['Date'] = pd.to_datetime(cleaned_nasdaq_predictions['Date'])

In [ ]:
# Create new dataframe of signals we want to include
nasdaq_signals = cleaned_nasdaq_predictions[['Date', 'Tweet content', 'prediction', 'ticker']].copy()

display(nasdaq_signals.head())

def center_prediction(prediction):
  if prediction == 0.5:
    return 0
  elif prediction > 0.5:
    return (prediction - 0.5)
  else:
    return -(.5-prediction)

# Cleanup prediction column
nasdaq_signals['prediction'] = nasdaq_signals['prediction'].apply(lambda x : center_prediction(x))
nasdaq_signals.set_index('Date')

In [ ]:
# Groupby date and just do a simple sum.....
# Could do it this way, i.e. with just a simple sum or by creating a threshold of the prediciton and then label each as 1 or 0
short_window = 12
long_window = 26
signal_window = 9

nasdaq_signals = pd.DataFrame(nasdaq_signals.resample('D', on='Date').prediction.sum())
# signals[f'prediction-{short_window}'] = signals['prediction'].rolling(short_window).mean()
# signals[f'prediction-{long_window}'] = signals['prediction'].rolling(long_window).mean()

# MACD calc
exp1 = nasdaq_signals['prediction'].ewm(span=12, adjust=False).mean()
exp2 = nasdaq_signals['prediction'].ewm(span=26, adjust=False).mean()
nasdaq_signals['MACD'] = macd = exp1 - exp2
nasdaq_signals['signal'] = exp3 = macd.ewm(span=signal_window, adjust=False).mean()

In [ ]:
nasdaq_signals = nasdaq_signals.dropna()
nasdaq_signals

## FinViz BeautifulSoup Scraper

In [ ]:
# # BeautifulSoup scraper example to adapt
# # https://www.tandfonline.com/doi/full/10.1080/24751839.2021.1874252
# website_url = 'https://finviz.com/quote.ashx?t='

# company_tikcers = ['ATVI']
# news_tables = {}
# parsed_data = []
# params = {}

# for ticker in company_tikcers:
#   url = website_url + ticker
#   res = requests.get(url, params=params)
#   response = res.dump
#   html = BeautifulSoup(response, 'html')
#   news_data = html.find(id='news-table')
#   news_tables[ticker] = news_data

# for ticker, news_table in news_tables.items():
#   for row in news_table.findAll('tr'):
#     title = row.a.text
#     date_data = row.td.text.split('')
#     if len(date_data) == 1:
#       time = date_data[0][0:7]
#     else:
#       date = datetime.datetime.strptime(date_data[0], '%b-%d-%y').strftime('%Y/%m/%d')
#       time = date_data[1][0:7]
#       parsed_data.append([ticker, date, time, title])

# dataset = pd.DataFrame(parsed_data, columns=['Company', 'Date', 'Time', 'News Headline'])



## Alpaca API- Stock Price

In [ ]:
# Upload your env file containing your API secret key.
from google.colab import files
uploaded = files.upload()

In [ ]:
load_dotenv()

In [ ]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Check the values were imported correctly by evaluating the type of each
display(type(alpaca_api_key))
display(type(alpaca_secret_key))


In [ ]:
# Create your Alpaca API REST object by calling Alpaca's tradeapi.REST function
# Set the parameters to your alpaca_api_key, alpaca_secret_key and api_version="v2" 
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [ ]:
# Create the list for the required tickers
tickers = ["ATVI", "SPY"]
nasdaq100_set = {'aal',
 'aapl',
 'adbe',
 'adp',
 'akam',
 'alxn',
 'amat',
 'amgn',
 'amzn',
 'atvi',
 'avgo',
 'bbby',
 'bidu',
 'bmrn',
 'ca',
 'celg',
 'cern',
 'chkp',
 'chtr',
 'cmcsa',
 'cost',
 'csco',
 'csx',
 'ctrp',
 'ctsh',
 'disca',
 'disck',
 'dish',
 'dltr',
 'ea',
 'ebay',
 'endp',
 'esrx',
 'expe',
 'fast',
 'fb',
 'fisv',
 'fox',
 'foxa',
 'gild',
 'goog',
 'googl',
 'hsic',
 'ilmn',
 'inct',
 'incy',
 'intu',
 'isrg',
 'jd',
 'khc',
 'lbtya',
 'lbtyk',
 'lltc',
 'lmca',
 'lmck',
 'lrcx',
 'lvnta',
 'mar',
 'mat',
 'mdlz',
 'mnst',
 'mu',
 'mxim',
 'myl',
 'nclh',
 'nflx',
 'ntes',
 'nvda',
 'nxpi',
 'orly',
 'payx',
 'pcar',
 'pcln',
 'pypl',
 'qcom',
 'qvca',
 'regn',
 'rost',
 'sbac',
 'sbux',
 'sndk',
 'srcl',
 'stx',
 'swks',
 'symc',
 'tmus',
 'trip',
 'tsco',
 'tsla',
 'txn',
 'ulta',
 'viab',
 'vod',
 'vrsk',
 'vrtx',
 'wdc',
 'wfm',
 'xlnx',
 'yhoo'}

In [ ]:
# Set the values for start_date and end_date using the pd.Timestamp function
# The start and end data should be 2019-05-01 to 2020-05-01
# Set the parameter tz to "America/New_York", 
# Set this all to the ISO format by calling the isoformat function 
start_date = pd.Timestamp("2020-09-10", tz= "America/New_York").isoformat()
end_date = pd.Timestamp("2021-09-10", tz= "America/New_York").isoformat()
timeframe = '1D'


In [ ]:
# Use the Alpaca get_barset function to gather the price information for each ticker
# Include the function parameters: tickers, timeframe, start, and end
# Be sure to call the df property to ensure that the returned information is set as a DataFrame
prices_df = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000
).df


# Review the first five rows of the resulting DataFrame 
# YOUR CODE HERE 
display(prices_df.head())
display(prices_df.tail())

In [ ]:
# prices_df.to_csv('/content/gdrive/MyDrive/twitter-sentiment/ATVI-SPY-year-prices', index=False)

In [ ]:
# ATVI_SPY_prices = pd$.read_csv('/content/gdrive/MyDrive/twitter-sentiment/ATVI-SPY-year-prices')
# ATVI_SPY_prices

In [ ]:
# Create an empty DataFrame for holding the closing prices
closing_prices_df = pd.DataFrame()

# Using a for loop, for every ticker in the tickers list, 
# Select the close price for each ticker in the prices_df Dataframe
# That will be set equal to closing_prices_df for the same ticker value
for ticker in tickers:
    closing_prices_df[ticker] = prices_df[ticker]['close']

# For the new closing_prices_df DataFrame, keep only the date component
closing_prices_df.index = closing_prices_df.index.date

# View the first and last five rows of the closing_prices_df DataFrame
display(closing_prices_df.head())
display(closing_prices_df.tail())

In [ ]:
# TOOLTIPS = [
#     ("index", "$index"),
#     ("(x,y)", "($x, $y)"),
#     ("close", "$@{ATVI}"),
# ]

# # source = ColumnDataSource(data={
# #     'id'      : pulled_tweets['id'],
# #     'tweet' : pulled_tweets['tweet']
# # })

# # Plot close price and tweets
# p = figure(
#     title="Price and Tweets",
#     x_axis_label='Date',
#     x_axis_type='datetime',
#     y_axis_label='Price',
#     plot_width=1000,
#     plot_height=400,
#     tooltips = TOOLTIPS)
# p.line(
#     closing_prices_df.index,
#     closing_prices_df['ATVI'],
#     legend_label=f'ATVI - close price',
#     line_color='green')
# p.asterisk(pulled_tweets.index, closing_prices_df['ATVI'], size=20, color="#F0027F")
# # p.line(signals.index, signals['signal'], legend_label=f"{signal_window} Day Rolling Signal Line")
# show(p)

In [ ]:
# Compute daily returns of the closing_prices_df DataFrame using the pct_change function 
# Be sure to drop the first row of NaN values
daily_returns_df = closing_prices_df.pct_change().dropna()
# Review the first and last five rows of the daily_returns_df DataFrame
display(daily_returns_df.head())
display(daily_returns_df.tail())

In [ ]:
# Using the Pandas describe function, generate summary statistics
# for each of the tickers in the daily_returns_df DataFrame
# YOUR CODE HERE 
daily_returns_df.describe()

In [ ]:
daily_returns_df.plot.density(figsize=(15,7), title="Daily Returns for SPY and ATVI")

# Basic Long Short Trade Bot

In [ ]:
# Concat together the basic version of a trading bot using twitter sentiment
df_trades_one = pd.concat([signals, daily_returns_df["ATVI"]], axis="columns").copy().dropna()

# Create our long and short signal, I used a reversion to mean strategy that seemed to work very well on this base level, will have to test on a few other tickers to be sure
df_trades_one["trade_signal"] = 0

df_trades_one.loc[( (df_trades_one["MACD"] - df_trades_one["signal"]) <= 0 ), "trade_signal"] = 1
df_trades_one.loc[( (df_trades_one["MACD"] - df_trades_one["signal"]) > 0 ), "trade_signal"] = -1

# Calculate our portfolio returns as well as buy and hold
df_trades_one["Strategy Daily Returns"] = df_trades_one["ATVI"] * df_trades_one["trade_signal"].shift()

df_trades_one["ATVI Strategy Returns"] = (1 + df_trades_one["Strategy Daily Returns"]).cumprod()

df_trades_one = df_trades_one.dropna()

df_trades_one["ATVI Buy and Hold"] = (1 + df_trades_one["ATVI"]).cumprod()




df_trades_one
# df_trades_one["trade_signal"].value_counts() this strategy was short about half the time and long about half the time 128 days long and 121 days short

# Performance Metrics

In [ ]:
# This is function takes in our price data and calculates the Compound Annual Growth Rate, Maximum Percent Draw Down, 
# the MAR Ratio (a meaure of annual growth over worst pullback) and the sharpe ratio for any one ticker. It then outputs the data
# in a pandas dataframe with the metrics as the index and the ticker as the column.

# to use the function you have to pass in a dataframe of just the closing prices with the column labled with the tickers
# and a list of tickers corresponding to those column names

def calculate_perf_stats(ticker_prices_df, ticker_list):
 
  # Establish the Dataframe that will contain all of the output information

  row_list = ["CAGR", "Max Drawdown", "Annual Volatility", "MAR Ratio", "Sharpe Ratio"]
  output_df = pd.DataFrame(index=row_list, columns=ticker_list)

  
  # Create a for loop to go through each ticker in the ticker list
  for ticker in ticker_list:

    # establish the dataframe that will be used in this function
    
    df_prices = pd.DataFrame()
    df_prices[ticker] = ticker_prices_df[ticker].copy()
    df_prices.index = ticker_prices_df.index

    df_prices[f"{ticker}_daily_return"] = df_prices[ticker].pct_change().dropna()

    # Calculate Compound Annual Growth Rate (CAGR) for ticker

    days_traded = df_prices[ticker].count() + 1
    days_per_year = 252
    
    end_value = df_prices[ticker][-1]
    start_value = df_prices[ticker][0]

    CAGR = ((end_value / start_value)**(1/(days_traded/days_per_year))) - 1

    # Calculate Maximum Drawdown

    top_price = 0.0 

    top_dollar = []

    for value in df_prices[ticker]:
      if value > top_price:
        top_price = value
      
      top_dollar.append(top_price)

    df_prices["top_dollar"] = top_dollar

    df_prices["draw_down"] = abs((df_prices[ticker] - df_prices["top_dollar"]) / df_prices["top_dollar"])

    max_dd = df_prices["draw_down"].max()

    # Calculates the MAR Ratio which is the annual growth rate over largest drawdown exposure

    mar_ratio = (CAGR / max_dd)

    
    # Calculates the Annualized Return and standard deviation and combines them to calculate the sharpe ratio
    
    annual_ret = df_prices[f"{ticker}_daily_return"].mean() * days_per_year
    annual_vol = df_prices[f"{ticker}_daily_return"].std() * np.sqrt(days_per_year)

    sharpe = annual_ret / annual_vol
    
    # Add calculations to the output dataframe

    output_df[ticker]["CAGR"] = CAGR
    output_df[ticker]["Max Drawdown"] = max_dd
    output_df[ticker]["Annual Volatility"] = annual_vol
    output_df[ticker]["MAR Ratio"] = mar_ratio
    output_df[ticker]["Sharpe Ratio"] = sharpe

  return output_df





In [ ]:

# Here is the performance metrics dataframe for the buy and hold positions


buy_and_hold_df = calculate_perf_stats(closing_prices_df, ["SPY", "ATVI"])

print("Buy and Hold performance Metrics")
buy_and_hold_df

In [ ]:
# Results from our first Strategy test

first_trade_test_df = calculate_perf_stats(df_trades_one, ["ATVI Buy and Hold", "ATVI Strategy Returns"])

first_trade_test_df

In [ ]:
# Performance Chart
df_trades_one[["ATVI Buy and Hold", "ATVI Strategy Returns"]].plot(figsize=(15,7), title="Cumulative Returns")